<a href="https://colab.research.google.com/github/Nick-McGrath/Novel_protein_generation/blob/main/Protein_backbone_generation_RFdiffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Generate protein backbones with RFdiffusion
- Clone Github RFD respoitory
- Explore possible changeable parameters
- Test limits and google colab processing power (50->100->1000->10,000 backbones at once)


Clone and set up RFdiffusion

In [1]:
#Install dependencies and clone repo
!rm -rf RFdiffusion
!pip install -q dm-haiku ml-collections py3Dmol biopython einops optax icecream omegaconf hydra-core

!git clone https://github.com/RosettaCommons/RFdiffusion.git
%cd RFdiffusion

#Download model weights
%cd RFdiffusion
!mkdir models && cd models
!wget http://files.ipd.uw.edu/pub/RFdiffusion/6f5902ac237024bdd0c176cb93063dc4/Base_ckpt.pt
!wget http://files.ipd.uw.edu/pub/RFdiffusion/e29311f6f1bf1af907f9ef9f44b8328b/Complex_base_ckpt.pt
!wget http://files.ipd.uw.edu/pub/RFdiffusion/60f09a193fb5e5ccdc4980417708dbab/Complex_Fold_base_ckpt.pt
!wget http://files.ipd.uw.edu/pub/RFdiffusion/74f51cfb8b440f50d70878e05361d8f0/InpaintSeq_ckpt.pt
!wget http://files.ipd.uw.edu/pub/RFdiffusion/76d00716416567174cdb7ca96e208296/InpaintSeq_Fold_ckpt.pt
!wget http://files.ipd.uw.edu/pub/RFdiffusion/5532d2e1f3a4738decd58b19d633b3c3/ActiveSite_ckpt.pt
!wget http://files.ipd.uw.edu/pub/RFdiffusion/12fc204edeae5b57713c5ad7dcb97d39/Base_epoch8_ckpt.pt
!wget http://files.ipd.uw.edu/pub/RFdiffusion/f572d396fae9206628714fb2ce00f72e/Complex_beta_ckpt.pt

# original structure prediction weights
!wget http://files.ipd.uw.edu/pub/RFdiffusion/1befcb9b28e2f778f53d47f18b7597fa/RF_structure_prediction_weights.pt

Cloning into 'RFdiffusion'...
remote: Enumerating objects: 471, done.
remote: Counting objects: 100% (321/321), done.
remote: Compressing objects: 100% (163/163), done.
remote: Total 471 (delta 208), reused 158 (delta 158), pack-reused 150 (from 1)
Receiving objects: 100% (471/471), 7.86 MiB | 22.75 MiB/s, done.
Resolving deltas: 100% (234/234), done.
/content/RFdiffusion
[Errno 2] No such file or directory: 'RFdiffusion'
/content/RFdiffusion
--2025-04-08 01:42:58--  http://files.ipd.uw.edu/pub/RFdiffusion/6f5902ac237024bdd0c176cb93063dc4/Base_ckpt.pt
Resolving files.ipd.uw.edu (files.ipd.uw.edu)... 128.95.160.134, 128.95.160.135, 2607:4000:406::160:135, ...
Connecting to files.ipd.uw.edu (files.ipd.uw.edu)|128.95.160.134|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 483616107 (461M) [application/octet-stream]
Saving to: ‘Base_ckpt.pt’

Base_ckpt.pt        100%[===================>] 461.21M  2.60MB/s    in 89s     

2025-04-08 01:44:27 (5.17 MB/s) - ‘Base_ckp

In [5]:
!conda env create -f env/SE3nv.yml

!conda activate SE3nv
%cd env/SE3Transformer
!pip install --no-cache-dir -r requirements.txt
!python setup.py install
%cd ../.. # change into the root directory of the repository
!pip install -e . # install the rfdiffusion module from the root of the repository

/bin/bash: line 1: conda: command not found
/bin/bash: line 1: conda: command not found
/content/RFdiffusion/env/SE3Transformer
  Cloning https://github.com/NVIDIA/dllogger to /tmp/pip-install-vo8x0uau/dllogger_01ab100e66d249679e82777b9460aca8
  Running command git clone --filter=blob:none --quiet https://github.com/NVIDIA/dllogger /tmp/pip-install-vo8x0uau/dllogger_01ab100e66d249679e82777b9460aca8
  Resolved https://github.com/NVIDIA/dllogger to commit 0478734ff7be75adde8d160e04872664d1c62e5f
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.4/97.4 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 382.1/382.1 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 216.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2

running install
/usr/local/lib/python3.11/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.11/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https

In [7]:
!conda activate SE3nv

/bin/bash: line 1: conda: command not found
